In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OrdinalEncoder

# 1. データ読み込み
print("Loading data...")
train_df = pd.read_csv('../data/train.csv')

# 2. 特徴量エンジニアリング (余計なことはしない、重要なことだけやる)
def preprocess(df):
    df = df.copy()
    
    # --- 数値変数の処理 ---
    # pdays: -1 は「過去の接触なし」。これを数値としてそのまま扱うと距離がおかしくなるが、
    # ツリーモデルは分割できるのでそのままでも良い。
    # しかし、明示的に「接触なしフラグ」を作ると親切。
    df['pdays_never'] = (df['pdays'] == -1).astype(int)
    
    # duration: 対数変換などを試してもいいが、ツリーモデルはそのままでも非線形性を捉える。
    # 重要なのは相互作用。
    # 1回の接触あたりの平均時間のような指標
    df['duration_per_contact'] = df['duration'] / (df['campaign'] + 1e-5)
    
    # balance: 負の値があるため、対数変換する場合は注意が必要だが、
    # ここでは生の値のまま扱う（ツリーモデルはスケールに依存しないため）。
    
    # --- カテゴリ変数の処理 ---
    # One-Hot Encodingはメモリを食う上に、ツリーの深さを無駄に消費する。
    # Ordinal Encoding（数値への置き換え）で十分、あるいはTarget Encodingがベスト。
    # ここではシンプルかつ強力なOrdinal Encodingを採用する。
    # カテゴリカルとして扱うカラム
    cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
    
    # 欠損値や未知のカテゴリ('unknown')はそのままで意味を持つので、特別扱いしない。
    
    return df, cat_cols

train_processed, cat_cols = preprocess(train_df)

# カテゴリ変数を数値IDに変換 (Ordinal Encoding)
# HistGradientBoostingClassifierはカテゴリ特徴量を指定すればネイティブに扱えるが、
# 文字列のままではエラーになるため数値化する。
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train_processed[cat_cols] = oe.fit_transform(train_processed[cat_cols])

# 3. モデリング & 評価 (Stratified K-Fold)
X = train_processed.drop(columns=['id', 'y'])
y = train_processed['y']
categorical_features_indices = [X.columns.get_loc(c) for c in cat_cols]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
scores = []

print("\nStarting Cross-Validation (5 Folds)...")
print("-" * 50)

for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # モデル構築
    # class_weightはあえて設定しない。AUCには不要なことが多い。
    # どうしてもRecallを上げたい場合のみ検討すべき。
    model = HistGradientBoostingClassifier(
        max_iter=1000,
        learning_rate=0.05,
        max_depth=10,
        early_stopping=True,
        categorical_features=categorical_features_indices,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    # 予測（確率値）
    val_preds = model.predict_proba(X_val)[:, 1]
    oof_preds[val_index] = val_preds
    
    score = roc_auc_score(y_val, val_preds)
    scores.append(score)
    print(f"Fold {fold+1} AUC: {score:.5f}")

print("-" * 50)
print(f"Mean AUC: {np.mean(scores):.5f}")
print(f"OOF AUC : {roc_auc_score(y, oof_preds):.5f}")

print("\n[Adviser's Note]")
print("この Mean AUC があなたの以前のスコアを上回っていれば、私の理論が正しい証明です。")
print("テストデータ(test.csv)があるなら、このモデルで全学習データを使って再学習し、predict_probaで予測して提出してください。")

Loading data...

Starting Cross-Validation (5 Folds)...
--------------------------------------------------
Fold 1 AUC: 0.93396
Fold 2 AUC: 0.92553
Fold 3 AUC: 0.92482
Fold 4 AUC: 0.93502
Fold 5 AUC: 0.93824
--------------------------------------------------
Mean AUC: 0.93151
OOF AUC : 0.93119

[Adviser's Note]
この Mean AUC があなたの以前のスコアを上回っていれば、私の理論が正しい証明です。
テストデータ(test.csv)があるなら、このモデルで全学習データを使って再学習し、predict_probaで予測して提出してください。


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_auc_score
# LightGBMが使えるならLightGBMを使ってください。ここでは標準ライブラリのHistGradientBoostingClassifierを使います。
# 性能はほぼ同等です。
from sklearn.ensemble import HistGradientBoostingClassifier

# --- 1. データ読み込み ---
# あなたの環境に合わせてパスを調整してください
train = pd.read_csv('../data/train.csv')
# test = pd.read_csv('test.csv') 
# ※ここではテストデータがないため、trainの一部をtestとして擬似的に作成して動作させます
# 本番では上の行のコメントアウトを外し、下の3行を削除してください
test = train.sample(1000, random_state=42).drop(columns=['y']).copy()
test['id'] = range(30000, 31000) # ダミーID
train = train.drop(test.index, errors='ignore').reset_index(drop=True)

print(f"Train shape: {train.shape}, Test shape: {test.shape}")

# データ結合（前処理を一括で行うため）
train['is_train'] = 1
test['is_train'] = 0
test['y'] = np.nan # テストデータのターゲットは不明

# 結合
df_all = pd.concat([train, test], ignore_index=True)

# --- 2. 高度な特徴量エンジニアリング ---

# (A) 日付・季節性の処理
month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
df_all['month_int'] = df_all['month'].map(month_map)
# 周期性（Monthの円環構造）
df_all['month_sin'] = np.sin(2 * np.pi * df_all['month_int'] / 12)
df_all['month_cos'] = np.cos(2 * np.pi * df_all['month_int'] / 12)

# (B) Duration（最重要変数）の強化
# 通話時間が0の場合は対数変換でエラーになるので+1
df_all['duration_log'] = np.log1p(df_all['duration'])
# キャンペーン回数あたりの通話時間（粘り強さ？）
df_all['duration_per_campaign'] = df_all['duration'] / (df_all['campaign'] + 1e-5)

# (C) 顧客プロファイルの擬似特定 (User Fingerprint)
# これらが一致すれば「同一人物」である可能性が高い
user_cols = ['age', 'job', 'marital', 'education', 'housing', 'loan']
df_all['user_hash'] = df_all[user_cols].astype(str).sum(axis=1)
# このユーザーがデータセットに何回登場するか？
df_all['user_count'] = df_all.groupby('user_hash')['id'].transform('count')

# (D) Aggregation Features (集約特徴量) - ここがスコアアップの肝です
# 「その職業(job)の平均通話時間は？」、「その教育レベル(education)の平均残高は？」
# これと自分の値との差分（Diff）や比率（Ratio）をとります。

group_cols = ['job', 'education', 'marital', 'contact']
target_cols = ['duration', 'balance', 'pdays']

for g_col in group_cols:
    for t_col in target_cols:
        # 平均
        mean_col = df_all.groupby(g_col)[t_col].transform('mean')
        df_all[f'{t_col}_mean_by_{g_col}'] = mean_col
        # 自分の値との比率（自分の通話時間は、同じ職業の平均より長いか？）
        df_all[f'{t_col}_ratio_by_{g_col}'] = df_all[t_col] / (mean_col + 1e-5)

# (E) pdays（経過日数）の特別な処理
# -1 は「連絡なし」。これを数値として混ぜると分布が歪む。
# 「初めての客か？」フラグを作成
df_all['is_new_client'] = (df_all['pdays'] == -1).astype(int)

# --- 3. カテゴリ変数のエンコーディング ---
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Ordinal Encoding (LightGBM/HistGradientBoostingはこれで十分)
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_all[cat_cols] = oe.fit_transform(df_all[cat_cols].astype(str))

# --- 4. データの分割 ---
train_processed = df_all[df_all['is_train'] == 1].drop(columns=['is_train', 'user_hash'])
test_processed = df_all[df_all['is_train'] == 0].drop(columns=['is_train', 'y', 'user_hash'])

X = train_processed.drop(columns=['id', 'y'])
y = train_processed['y']
X_test = test_processed.drop(columns=['id'])

# カテゴリカル変数のインデックスを取得
cat_indices = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]

# --- 5. モデリング (Stratified K-Fold + HistGradientBoosting) ---
# LightGBMを使用する場合は lgb.LGBMClassifier に書き換えてください
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds_accum = np.zeros(len(X_test))

print("\nStarting Training...")
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # モデル設定：0.99を目指すなら過学習スレスレまで攻める設定もありだが、
    # まずは堅実にEarly Stoppingを入れる
    model = HistGradientBoostingClassifier(
        max_iter=2000,
        learning_rate=0.03, # ゆっくり学習させる
        max_depth=12,      # 少し深めに
        l2_regularization=1.0, # 過学習抑制
        categorical_features=cat_indices,
        early_stopping=True,
        random_state=42
    )
    
    model.fit(X_tr, y_tr)
    
    # 予測
    val_pred = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_pred
    
    score = roc_auc_score(y_val, val_pred)
    print(f"Fold {fold+1} AUC: {score:.5f}")
    
    # テストデータへの予測（平均をとるため加算）
    test_preds_accum += model.predict_proba(X_test)[:, 1]

# 最終スコア
mean_auc = roc_auc_score(y, oof_preds)
print("-" * 30)
print(f"OOF AUC: {mean_auc:.5f}")

# --- 6. 提出ファイル作成 ---
test_preds_avg = test_preds_accum / skf.get_n_splits()

submission = pd.DataFrame({
    'id': test_processed['id'],
    'y': test_preds_avg
})

# submission.to_csv('submission_advanced.csv', index=False)
print("Submission file created (in memory).")
print(submission.head())

Train shape: (26128, 18), Test shape: (1000, 17)

Starting Training...
Fold 1 AUC: 0.92970
Fold 2 AUC: 0.93088
Fold 3 AUC: 0.92638
Fold 4 AUC: 0.93212
Fold 5 AUC: 0.93220
------------------------------
OOF AUC: 0.93020
Submission file created (in memory).
          id         y
26128  30000  0.011198
26129  30001  0.011907
26130  30002  0.005742
26131  30003  0.018738
26132  30004  0.037102


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer


# 1. 設定 & データ読み込み
TRAIN_PATH = '../data/train.csv'
TEST_PATH = '../data/test.csv'  # テストデータのファイル名

print("Loading data...")
train = pd.read_csv(TRAIN_PATH)

# --- テストデータの読み込み ---
try:
    test = pd.read_csv(TEST_PATH)
    print(f"Test data loaded. Shape: {test.shape}")
except FileNotFoundError:
    print("【警告】test.csvが見つかりません。学習用データの一部を代用してシミュレーションします。")
    print("本番提出用には、必ず正しいtest.csvを用意してください。")
    test = train.sample(2000, random_state=99).drop(columns=['y']).copy()
    test['id'] = range(30000, 32000)
    train = train.drop(test.index, errors='ignore').reset_index(drop=True)

# データ結合
train['is_train'] = 1
test['is_train'] = 0
test['y'] = np.nan
df_all = pd.concat([train, test], ignore_index=True)


# 2. 特徴量エンジニアリング 
print("Engineering features...")

# (A) 数値データの変換
df_all['duration_log'] = np.log1p(df_all['duration'])
# キャンペーン効率
df_all['efficiency'] = df_all['duration'] / (df_all['campaign'] + 1e-5)

# (B) 強力な比率特徴量
# 「その職業・年齢層の中で、今回の通話時間は異常か？」
# 異常に長い＝脈あり、のシグナルを強調
df_all['age_bin'] = pd.cut(df_all['age'], bins=5, labels=False)
group_cols = ['job', 'housing', 'age_bin']
target_cols = ['duration']

for g_col in group_cols:
    for t_col in target_cols:
        # 平均との比率
        mean_val = df_all.groupby(g_col)[t_col].transform('mean')
        df_all[f'{t_col}_ratio_{g_col}'] = df_all[t_col] / (mean_val + 1e-5)

# (C) 周期性 (Day/Month)
# 給料日（25日など）や月末月初の影響を捉える
month_map = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
             'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
df_all['month_int'] = df_all['month'].map(month_map)
df_all['month_sin'] = np.sin(2 * np.pi * df_all['month_int'] / 12)
df_all['month_cos'] = np.cos(2 * np.pi * df_all['month_int'] / 12)

df_all['day_sin'] = np.sin(2 * np.pi * df_all['day'] / 31)
df_all['day_cos'] = np.cos(2 * np.pi * df_all['day'] / 31)

# (D) 過去の成功体験 (最強の予測因子)
# poutcome='success' はほぼ勝ち確。これを明示的にフラグ化。
df_all['is_success_prev'] = (df_all['poutcome'] == 'success').astype(int)
df_all['pdays_clean'] = df_all['pdays'].replace(-1, 999)

# (E) カテゴリカル変数の処理
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df_all[cat_cols] = oe.fit_transform(df_all[cat_cols].astype(str))

# データの再分割
X = df_all[df_all['is_train'] == 1].drop(columns=['id', 'y', 'is_train'])
y = df_all[df_all['is_train'] == 1]['y']
X_test = df_all[df_all['is_train'] == 0].drop(columns=['id', 'y', 'is_train'])

# カテゴリ変数のインデックス（HGBC用）
cat_indices = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]

# 3. Hybrid Ensemble (GBDT + ExtraTrees)
# 異なるアルゴリズムを混ぜることでスコアを安定させる
print("\nStarting Hybrid Ensemble Training...")

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

# --- Model 1: HistGradientBoosting (Main) ---
# Seed Averaging
GBM_SEEDS = [42, 2024]
for seed in GBM_SEEDS:
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for train_idx, val_idx in skf.split(X, y):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = HistGradientBoostingClassifier(
            max_iter=1000, learning_rate=0.03, max_depth=8,
            categorical_features=cat_indices, early_stopping=True, random_state=seed
        )
        model.fit(X_tr, y_tr)
        
        oof_preds[val_idx] += model.predict_proba(X_val)[:, 1] / len(GBM_SEEDS) / 2 # /2は後で他モデルと足すため
        test_preds += model.predict_proba(X_test)[:, 1] / 5 / len(GBM_SEEDS) / 2

print(f"GBDT Part Done. Current AUC (Half): {roc_auc_score(y, oof_preds*2):.5f}")

# --- Model 2: ExtraTreesClassifier (Diversity) ---
# 欠損値埋めが必要。
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_test_imputed = imputer.transform(X_test)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=99)
et_oof = np.zeros(len(X))
et_test = np.zeros(len(X_test))

for train_idx, val_idx in skf.split(X_imputed, y):
    X_tr, X_val = X_imputed[train_idx], X_imputed[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # 決定木をランダムに深く作るモデル
    model = ExtraTreesClassifier(n_estimators=500, max_depth=20, min_samples_leaf=10, n_jobs=-1, random_state=99)
    model.fit(X_tr, y_tr)
    
    et_oof[val_idx] = model.predict_proba(X_val)[:, 1]
    et_test += model.predict_proba(X_test_imputed)[:, 1] / 5

# アンサンブル (GBDT: 70%, ExtraTrees: 30% くらいが相場だが、ここでは等倍ブレンドで安定を狙う)
final_oof = oof_preds * 2 * 0.7 + et_oof * 0.3
final_test_preds = test_preds * 2 * 0.7 + et_test * 0.3

print(f"Hybrid Ensemble OOF AUC: {roc_auc_score(y, final_oof):.5f}")

# 4. Dynamic Pseudo-Labeling (強制注入)
print("\nExecuting Dynamic Pseudo-Labeling...")

# 上位5%と下位5%を強制的に教師データにする
# これなら「自信があるデータがない」という事態を防げる
n_top = int(len(X_test) * 0.05)
n_bottom = int(len(X_test) * 0.05)

# インデックスを取得
sorted_indices = np.argsort(final_test_preds)
top_idx = sorted_indices[-n_top:]     # 予測確率が高いインデックス
bottom_idx = sorted_indices[:n_bottom] # 予測確率が低いインデックス

X_pseudo_high = X_test.iloc[top_idx]
y_pseudo_high = pd.Series(1, index=X_pseudo_high.index)

X_pseudo_low = X_test.iloc[bottom_idx]
y_pseudo_low = pd.Series(0, index=X_pseudo_low.index)

print(f"Adding {len(X_pseudo_high)} positive (Top 5%) and {len(X_pseudo_low)} negative pseudo-labels.")

# データ拡張
X_augmented = pd.concat([X, X_pseudo_high, X_pseudo_low])
y_augmented = pd.concat([y, y_pseudo_high, y_pseudo_low])

# 最終学習 (GBDTで全データ学習)
final_model = HistGradientBoostingClassifier(
    max_iter=1500, learning_rate=0.03, max_depth=8,
    categorical_features=cat_indices, early_stopping=False, random_state=42
)

final_model.fit(X_augmented, y_augmented)
submission_preds = final_model.predict_proba(X_test)[:, 1]

# 念のため、アンサンブル結果とブレンド（暴走防止）
submission_preds = 0.6 * submission_preds + 0.4 * final_test_preds

# 5. 提出ファイル作成
submission = pd.DataFrame({
    'id': test['id'],
    'y': submission_preds
})

submission.to_csv('submission_final_strategy.csv', index=False)
print("\nSubmission file created: 'submission_final_strategy.csv'")
print(submission.head())

Loading data...
Test data loaded. Shape: (18083, 17)
Engineering features...

Starting Hybrid Ensemble Training...
GBDT Part Done. Current AUC (Half): 0.93237
Hybrid Ensemble OOF AUC: 0.93386

Executing Dynamic Pseudo-Labeling...
Adding 904 positive (Top 5%) and 904 negative pseudo-labels.

Submission file created: 'submission_final_strategy.csv'
   id         y
0   1  0.827964
1   2  0.709314
2   3  0.000506
3   4  0.000643
4   5  0.116926
